In [1]:
import warnings

import numpy as np

from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck18 as cosmo
from astropy.io import fits
from astropy.table import Table, vstack
import astropy.units as u
from astropy.utils.metadata import MergeConflictWarning

import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
from lsst.geom import degrees, SpherePoint

In [2]:
repo = "/repo/embargo"
collection = "u/elhoward/TOO/S251112cm/d_2025_11_15/run_02"
# collection = "LSSTCam/raw/all"
exposure = 2025072800100

butler = Butler(repo, collections=collection)

In [3]:
dataref_iterator = butler.registry.queryDatasets(datasetType="difference_image", collections=[collection])

In [4]:
data_refs = [dr for dr in dataref_iterator.expanded()]

In [5]:
len(data_refs)

32

In [6]:
# We're mixing columns from dia_src and dia_src_table
# This is dangerous.  I don't think there is a guarantee that the ordering is the same.
# But I want the flags from dia_src and the science flux from dia_src_table

"""
good = ~dia_src["slot_Shape_flag"] & \
    (dia_src["base_PsfFlux_instFlux"] / dia_src["base_PsfFlux_instFluxErr"] > snr_threshold) & \
    ~dia_src["base_PixelFlags_flag_edge"] & \
    ((dia_src_table["scienceFlux"] / dia_src_table["scienceFluxErr"]) < max_science_snr) & \
    ~dia_src_table["pixelFlags_streak"]
"""

def good_src(cat, snr_threshold=7.5, max_science_snr=200):
    """Good for the dia_source_unfiltered"""
    good = ~cat["slot_Shape_flag"] & \
        (cat["base_PsfFlux_instFlux"] / cat["base_PsfFlux_instFluxErr"] > snr_threshold) & \
        ~cat["base_PixelFlags_flag_edge"]

    good_cat = cat[good].copy()

    return good_cat


def good_src_table(cat, snr_threshold=7.5, max_science_snr=200):
    """Good for the dia_source_table"""

    # If I were doing just dia_src_table, I'd do something like this:
    good = (cat["snr"] > snr_threshold) & \
        ~cat["shape_flag"] & \
        ~cat["pixelFlags_bad"] & \
        ~cat["pixelFlags_cr"] & \
        ((cat["scienceFlux"] / cat["scienceFluxErr"]) < max_science_snr)

    good_cat = cat[good].copy(deep=True)

    return good_cat

In [6]:
# Load NED-GWF table
url = "https://ned.ipac.caltech.edu/uri/NED::GWFglist/fits/S250725j/3"

ned_gwf = Table(fits.getdata(url))

In [7]:
ned_gwf

objname,ra,dec,objtype,z,z_unc,z_tech,z_qual,z_qual_flag,z_refcode,ziDist,ziDist_unc,ziDist_method,ziDist_indicator,ziDist_refcode,DistMpc,DistMpc_unc,DistMpc_method,ebv,A_FUV_MWext,A_NUV_MWext,A_J_MWext,A_H_MWext,A_Ks_MWext,A_W1_MWext,A_W2_MWext,A_W3_MWext,A_W4_MWext,m_FUV,m_FUV_unc,m_NUV,m_NUV_unc,Lum_FUV,Lum_FUV_unc,Lum_NUV,Lum_NUV_unc,GALEXphot,m_J,m_J_unc,m_H,m_H_unc,m_Ks,m_Ks_unc,Lum_J,Lum_J_unc,Lum_H,Lum_H_unc,Lum_Ks,Lum_Ks_unc,tMASSphot,m_W1,m_W1_unc,m_W2,m_W2_unc,m_W3,m_W3_unc,m_W4,m_W4_unc,Lum_W1,Lum_W1_unc,Lum_W2,Lum_W2_unc,Lum_W3,Lum_W3_unc,Lum_W4,Lum_W4_unc,WISEphot,SFR_W4,SFR_W4_unc,SFR_hybrid,SFR_hybrid_unc,ET_flag,Mstar,Mstar_unc,MLratio,P_2D,P_3D,P_2D_csum,P_3D_csum,P_Mstar,P_SFR,P_sSFR,P_LumW1,P_3D_Mstar,P_3D_SFR,P_3D_sSFR,P_3D_LumW1
str30,float64,float64,str6,float32,float32,str4,bool,bool,str19,float64,float64,str30,str1,str19,float32,float32,str30,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,bool,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ESO 387- G 034,231.6528204,-37.3724692,G,0.037783,0.00015,SPEC,False,False,20096dF...C...0000J,nan,nan,,,,167.44652,0.6461045,Redshift,0.17766857,1.422655,1.5966593,0.14433374,0.089913115,0.061669808,0.035265088,0.024600904,0.009383547,0.004881834,nan,nan,nan,nan,nan,nan,nan,nan,,13.198,0.066,12.204,0.062,11.758,0.072,7.795419342302675e+43,4.73885628202686e+42,8.841351269931583e+43,5.048943518722763e+42,6.511709613223098e+43,4.318347683425701e+42,XSC,11.36,0.008,11.304,0.022,8.76,0.061,7.676,0.2,2.7411061382018707e+43,2.0197889868418386e+41,1.1552955018882654e+43,2.3410244221990544e+41,8.70869127037572e+42,4.892973035864577e+41,3.2526479081463914e+42,5.9917985104499694e+41,APER,0.6354482758816863,0.11705779845890799,nan,nan,False,86743865132.97058,639173730.01324,0.5,74.03874388086523,1.9368365216076273e-07,0.7235941405618947,0.8884157865603718,0.0007304494491976644,0.0003651452617267918,0.0021539295928469163,0.0007283701155649778,1.4147611703942117e-10,7.072266786044262e-11,4.171809500397354e-10,1.4107338410738172e-10
WISEA J153059.36-345329.1,232.7473713,-34.8914357,G,0.03885,0.000113,SPEC,False,False,20096dF...C...0000J,nan,nan,,,,172.30984,0.48673204,Redshift,0.28381592,2.2726145,2.550577,0.23056532,0.14363134,0.098514184,0.05633407,0.039298613,0.0149897095,0.007798466,19.7465,99.0,19.9569,0.198175,2.6127594252917233e+43,nan,1.8593958507193688e+43,3.3939925473306354e+42,ASC,12.443,0.035,11.642,0.045,11.362,0.054,1.7914305479815155e+44,5.775082098464436e+42,1.6507219756593657e+44,6.841898372992947e+42,1.0273089680386861e+44,5.1095772624588054e+42,XSC,10.939,0.006,10.99,0.013,8.991,0.051,7.994,0.288,4.361352812380106e+43,2.410252969520371e+41,1.655923681091634e+43,1.9827768788763583e+41,7.493131458017681e+42,3.5198458364689606e+41,2.5767468451684202e+42,6.8352497260065955e+41,APER,0.5034019624580138,0.13353574614495825,1.5978803650435096,nan,False,138017494062.6616,762738281.4937882,0.5,212.53027944801423,1.7258226806519284e-07,0.25606633090214176,0.8975728263558312,0.0011622124788095735,0.00028926798342550933,0.00432605635630693,0.0011589040671211653,2.0057726556662607e-10,4.992252465821901e-11,7.46600617749294e-10,2.0000629237374718e-10
WISEA J152736.89-354551.9,231.903731,-35.7644294,G,0.039094,0.000103,SPEC,False,False,20096dF...C...0000J,nan,nan,,,,173.42303,0.4436584,Redshift,0.30226958,2.4203794,2.716415,0.24555664,0.15297022,0.10491955,0.0599969,0.0418538,0.015964337,0.008305521,nan,nan,nan,nan,nan,nan,nan,nan,,13.431,0.055,12.655,0.073,12.424,0.088,7.406111564199192e+43,3.751829499263927e+42,6.634418467785575e+43,4.4608321

Match dia_src table to this set of galaxies.  5 arcmin

In [11]:
def load_and_match_dia_src_against_cat(dr, cat, dataset_type="dia_source", radius=30 * u.arcsec, debug=False):
    dia_src = butler.get(dataset_type, dataId=dr.dataId).asAstropy()
    good_dia_src = good_src(dia_src)
    
    dia_coord = SkyCoord(good_dia_src["coord_ra"], good_dia_src["coord_dec"])
    cat_coord = SkyCoord(cat["ra"] * u.deg, cat["dec"] * u.deg)
    
    idx, sep2d, _ = dia_coord.match_to_catalog_sky(cat_coord)

    close_enough = sep2d < radius
    matching_idx = idx[close_enough]

    if debug:
        print(len(dia_coord))
        print(cat_coord[364])
        print(idx)
        print(sep2d)
        print(matching_idx)
        print("----")

    matched_dia_src = good_dia_src[close_enough].copy()
    matched_cat = cat[matching_idx].copy()

    return matched_dia_src, matched_cat
        

In [11]:
# Use an approximate matching radius given redshift.
z = 0.1
matching_radius_physical = 50 * u.kpc
matching_radius_angular = matching_radius_physical * cosmo.arcsec_per_kpc_proper(z)
print(matching_radius_physical, matching_radius_angular)

50.0 kpc 26.22622275160692 arcsec


In [12]:
# So let's call that 30"
ned_matching_radius = 30 * u.arcsec

In [13]:
ned_dr_matches = []
ned_dia_src_matches = []
ned_cat_matches = []

for dr in data_refs:
    matched_dia_src, matched_cat = load_and_match_dia_src_against_cat(dr, ned_gwf, radius=ned_matching_radius)
    ned_dr_matches.append(dr)
    ned_dia_src_matches.append(matched_dia_src)
    ned_cat_matches.append(matched_cat)

In [14]:
ned_dr_match = [dr for dr, dsm in zip(ned_dr_matches, ned_dia_src_matches) if len(dsm) > 0]
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=MergeConflictWarning)
    ned_dia_src_match = vstack(ned_dia_src_matches)
    ned_cat_match = vstack(ned_cat_matches)

In [15]:
ned_dr_match[0].dataId

{instrument: 'LSSTCam', detector: 183, visit: 2025072800100, band: 'i', day_obs: 20250728, physical_filter: 'i_39'}

In [16]:
ned_dia_src_match

id,coord_ra,coord_dec,parent,coord_raErr,coord_decErr,coord_ra_dec_Cov,is_negative,detect_isPrimary,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_PsfFlux_flag_badCentroid,ext_shapeHSM_HsmPsfMoments_flag_badCentroid,ext_shapeHSM_HsmSourceMoments_flag_badCentroid,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_PsfFlux_flag_badCentroid_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_notAtMaximum,base_SdssCentroid_flag_near_edge,base_CircularApertureFlux_flag_badCentroid_near_edge,base_PsfFlux_flag_badCentroid_near_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_near_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_near_edge,base_SdssCentroid_flag_resetToPeak,base_CircularApertureFlux_flag_badCentroid_resetToPeak,base_PsfFlux_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_resetToPeak,base_SdssCentroid_flag_badError,base_CircularApertureFlux_flag_badCentroid_badError,base_PsfFlux_flag_badCentroid_badError,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_badError,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_badError,ip_diffim_DipoleFit_pos_instFlux,slot_Centroid_pos_instFlux,ip_diffim_DipoleFit_pos_instFluxErr,slot_Centroid_pos_instFluxErr,ip_diffim_DipoleFit_neg_instFlux,slot_Centroid_neg_instFlux,ip_diffim_DipoleFit_neg_instFluxErr,slot_Centroid_neg_instFluxErr,ip_diffim_DipoleFit_instFlux,slot_Centroid_instFlux,ip_diffim_DipoleFit_instFluxErr,slot_Centroid_instFluxErr,ip_diffim_DipoleFit_pos_x,slot_Centroid_pos_x,ip_diffim_DipoleFit_pos_y,slot_Centroid_pos_y,ip_diffim_DipoleFit_neg_x,slot_Centroid_neg_x,ip_diffim_DipoleFit_neg_y,slot_Centroid_neg_y,ip_diffim_DipoleFit_x,slot_Centroid_x,ip_diffim_DipoleFit_y,slot_Centroid_y,ip_diffim_DipoleFit_xErr,slot_Centroid_xErr,ip_diffim_DipoleFit_yErr,slot_Centroid_yErr,ip_diffim_DipoleFit_orientation,slot_Centroid_orientation,ip_diffim_DipoleFit_separation,slot_Centroid_separation,ip_diffim_DipoleFit_chi2dof,slot_Centroid_chi2dof,ip_diffim_DipoleFit_nData,slot_Centroid_nData,ip_diffim_DipoleFit_signalToNoise,slot_Centroid_signalToNoise,ip_diffim_DipoleFit_classification,slot_Centroid_classification,ip_diffim_DipoleFit_classificationAttempted,slot_Centroid_classificationAttempted,ip_diffim_DipoleFit_flag,slot_Centroid_flag,ip_diffim_DipoleFit_flag_edge,slot_Centroid_flag_edge,ext_shapeHSM_HsmPsfMoments_flag,slot_PsfShape_flag,ext_shapeHSM_HsmPsfMoments_flag_no_pixels,slot_PsfShape_flag_no_pixels,ext_shapeHSM_HsmPsfMoments_flag_not_contained,slot_PsfShape_flag_not_contained,ext_shapeHSM_HsmPsfMoments_flag_parent_source,slot_PsfShape_flag_parent_source,ext_shapeHSM_HsmPsfMoments_flag_galsim,slot_PsfShape_flag_galsim,ext_shapeHSM_HsmPsfMoments_flag_invalid_param,slot_PsfShape_flag_invalid_param,ext_shapeHSM_HsmPsfMoments_flag_edge,slot_PsfShape_flag_edge,ext_shapeHSM_HsmPsfMoments_flag_no_psf,slot_PsfShape_flag_no_psf,ext_shapeHSM_HsmPsfMoments_x,slot_PsfShape_x,ext_shapeHSM_HsmPsfMoments_y,slo

In [17]:
ned_dia_src_match.write("dia_src_match_30arcsec.ecsv", overwrite=True)

In [18]:
ned_cat_match

objname,ra,dec,objtype,z,z_unc,z_tech,z_qual,z_qual_flag,z_refcode,ziDist,ziDist_unc,ziDist_method,ziDist_indicator,ziDist_refcode,DistMpc,DistMpc_unc,DistMpc_method,ebv,A_FUV_MWext,A_NUV_MWext,A_J_MWext,A_H_MWext,A_Ks_MWext,A_W1_MWext,A_W2_MWext,A_W3_MWext,A_W4_MWext,m_FUV,m_FUV_unc,m_NUV,m_NUV_unc,Lum_FUV,Lum_FUV_unc,Lum_NUV,Lum_NUV_unc,GALEXphot,m_J,m_J_unc,m_H,m_H_unc,m_Ks,m_Ks_unc,Lum_J,Lum_J_unc,Lum_H,Lum_H_unc,Lum_Ks,Lum_Ks_unc,tMASSphot,m_W1,m_W1_unc,m_W2,m_W2_unc,m_W3,m_W3_unc,m_W4,m_W4_unc,Lum_W1,Lum_W1_unc,Lum_W2,Lum_W2_unc,Lum_W3,Lum_W3_unc,Lum_W4,Lum_W4_unc,WISEphot,SFR_W4,SFR_W4_unc,SFR_hybrid,SFR_hybrid_unc,ET_flag,Mstar,Mstar_unc,MLratio,P_2D,P_3D,P_2D_csum,P_3D_csum,P_Mstar,P_SFR,P_sSFR,P_LumW1,P_3D_Mstar,P_3D_SFR,P_3D_sSFR,P_3D_LumW1
str30,float64,float64,str6,float32,float32,str4,bool,bool,str19,float64,float64,str30,str1,str19,float32,float32,str30,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,bool,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
WISEA J151946.60-370621.4,229.9442007,-37.105956,G,0.048273,0.00015,SPEC,False,False,20096dF...C...0000J,nan,nan,,,,215.57501,0.6461045,Redshift,0.08934936,0.71545184,0.8029585,0.0725853,0.04521722,0.031013688,0.017734779,0.012371771,0.004718977,0.0024550697,nan,nan,nan,nan,nan,nan,nan,nan,,12.641,0.061,12.107,0.054,11.774,0.084,2.0201538853117327e+44,1.1350223547533035e+43,1.5377472383597302e+44,7.648370035367608e+42,1.0338977580672846e+44,7.999208720855261e+42,XSC,11.217,0.009,11.211,0.017,10.72,0.222,8.859,0.539,5.099857119249498e+43,4.227568435957745e+41,2.0627463193030572e+43,3.2298689271176825e+41,2.363368665557964e+42,4.832530790260124e+41,1.8093007058682e+42,8.982343053338805e+41,APER,0.3534710938789138,0.1754820862204768,nan,nan,True,161387883520.5537,1337838112.644856,0.5,47.60553261925345,3.131411585699933e-07,0.8170270796366751,0.8406955597501661,0.0013590089678855912,0.0002031137702092098,0.007204269679538347,0.0013551403455501906,4.255616427107048e-10,6.360328132483134e-11,2.255953354101312e-09,4.243502178305277e-10
WISEA J151946.60-370621.4,229.9442007,-37.105956,G,0.048273,0.00015,SPEC,False,False,20096dF...C...0000J,nan,nan,,,,215.57501,0.6461045,Redshift,0.08934936,0.71545184,0.8029585,0.0725853,0.04521722,0.031013688,0.017734779,0.012371771,0.004718977,0.0024550697,nan,nan,nan,nan,nan,nan,nan,nan,,12.641,0.061,12.107,0.054,11.774,0.084,2.0201538853117327e+44,1.1350223547533035e+43,1.5377472383597302e+44,7.648370035367608e+42,1.0338977580672846e+44,7.999208720855261e+42,XSC,11.217,0.009,11.211,0.017,10.72,0.222,8.859,0.539,5.099857119249498e+43,4.227568435957745e+41,2.0627463193030572e+43,3.2298689271176825e+41,2.363368665557964e+42,4.832530790260124e+41,1.8093007058682e+42,8.982343053338805e+41,APER,0.3534710938789138,0.1754820862204768,nan,nan,True,161387883520.5537,1337838112.644856,0.5,47.60553261925345,3.131411585699933e-07,0.8170270796366751,0.8406955597501661,0.0013590089678855912,0.0002031137702092098,0.007204269679538347,0.0013551403455501906,4.255616427107048e-10,6.360328132483134e-11,2.255953354101312e-09,4.243502178305277e-10
WISEA J152606.75-371305.5,231.5281498,-37.2182162,G,0.072775,nan,PHOT,False,False,2014ApJS..210....9B,nan,nan,,,,330.69308,nan,Redshift,0.14877512,1.1912949,1.3370017,0.120861374,0.07529094,0.05164072,0.029530082,0.020600166,0.0078575425,0.004087923,nan,nan,nan,nan,nan,nan,nan,nan,,14.76,0.157,14.093,0.177,13.337,0.164,7.059084614645513e+43,1.0207942676747508e+43,5.972659449345846e+43,9.737135068721154e+42,5.877355213209346e+43,8.878015897439091e+42,XSC,13.641,0.091,13.677,0.109,11.

## Crossmatch to GCN candidates

In [47]:
names = ("AT name", "ra", "dec", "discovery_date (UT)", "filt", "mag", "mag_err")
rows = (
("AT2025adjp", 4.539622, -33.854284, "2025-11-14T02:08:26.016", "g", 21.13, 0.04),
("AT2025adkl", 7.535766, -41.078709, "2025-11-14T02:13:43.104", "g", 20.91, 0.03),
("AT2025adgp", 26.942508, -51.823756, "2025-11-13T03:09:48", "q", 19.31, 0.12),
("AT2025adgq", 7.38245842, -39.5143190, "2025-11-13T00:03:38", "q", 19.28, 0.10), 
("AT2025addc", 73.858875, -53.16211389, "2025-11-12T16:09:51", "L", 19.93, 0.20),
("AT2025adcv", 46.49416667, -54.15183333, "2025-11-12T16:09:51", "L", 19.30, 0.10),
("AT2025adcy", 68.869, -57.47545556, "2025-11-12T16:09:51", "L", 19.71, 0.12)
    )

gcn_table = Table(rows=rows, names=names)

In [49]:
gcn_table

AT name,ra,dec,discovery_date (UT),filt,mag,mag_err
str10,float64,float64,str23,str1,float64,float64
AT2025adjp,4.539622,-33.854284,2025-11-14T02:08:26.016,g,21.13,0.04
AT2025adkl,7.535766,-41.078709,2025-11-14T02:13:43.104,g,20.91,0.03
AT2025adgp,26.942508,-51.823756,2025-11-13T03:09:48,q,19.31,0.12
AT2025adgq,7.38245842,-39.514319,2025-11-13T00:03:38,q,19.28,0.1
AT2025addc,73.858875,-53.16211389,2025-11-12T16:09:51,L,19.93,0.2
AT2025adcv,46.49416667,-54.15183333,2025-11-12T16:09:51,L,19.3,0.1
AT2025adcy,68.869,-57.47545556,2025-11-12T16:09:51,L,19.71,0.12


In [50]:
# Match GCN candidates
gcn_matching_radius = 5 * u.arcsec

In [51]:
gcn_dr_matches = []
gcn_dia_src_matches = []
gcn_cat_matches = []

for dr in data_refs:
    matched_dia_src, matched_cat = load_and_match_dia_src_against_cat(dr, gcn_table, radius=gcn_matching_radius, dataset_type="dia_source_unfiltered")
    gcn_dr_matches.append(dr)
    gcn_dia_src_matches.append(matched_dia_src)
    gcn_cat_matches.append(matched_cat)

In [52]:
gcn_dr_match = [dr for dr, dsm in zip(gcn_dr_matches, gcn_dia_src_matches) if len(dsm) > 0]
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=MergeConflictWarning)
    gcn_dia_src_match = vstack(gcn_dia_src_matches)
    gcn_cat_match = vstack(gcn_cat_matches)

In [53]:
gcn_cat_match

AT name,ra,dec,discovery_date (UT),filt,mag,mag_err
str10,float64,float64,str23,str1,float64,float64
AT2025adgq,7.38245842,-39.514319,2025-11-13T00:03:38,q,19.28,0.1
AT2025addc,73.858875,-53.16211389,2025-11-12T16:09:51,L,19.93,0.2
AT2025adjp,4.539622,-33.854284,2025-11-14T02:08:26.016,g,21.13,0.04
AT2025adgq,7.38245842,-39.514319,2025-11-13T00:03:38,q,19.28,0.1
AT2025adjp,4.539622,-33.854284,2025-11-14T02:08:26.016,g,21.13,0.04
AT2025addc,73.858875,-53.16211389,2025-11-12T16:09:51,L,19.93,0.2


In [54]:
gcn_dr_match

[DatasetRef(DatasetType('difference_image', {band, instrument, day_obs, detector, physical_filter, visit}, ExposureF), {instrument: 'LSSTCam', detector: 105, visit: 2025111400089, band: 'g', day_obs: 20251114, physical_filter: 'g_6'}, run='u/elhoward/TOO/S251112cm/d_2025_11_15/run_02/20251116T041159Z', id=019a8add-6863-7a3d-8280-3a1dcf068f38),
 DatasetRef(DatasetType('difference_image', {band, instrument, day_obs, detector, physical_filter, visit}, ExposureF), {instrument: 'LSSTCam', detector: 147, visit: 2025111400234, band: 'i', day_obs: 20251114, physical_filter: 'i_39'}, run='u/elhoward/TOO/S251112cm/d_2025_11_15/run_02/20251116T041159Z', id=019a8add-6864-7645-9d98-ea7ebd6f73fc),
 DatasetRef(DatasetType('difference_image', {band, instrument, day_obs, detector, physical_filter, visit}, ExposureF), {instrument: 'LSSTCam', detector: 108, visit: 2025111400197, band: 'i', day_obs: 20251114, physical_filter: 'i_39'}, run='u/elhoward/TOO/S251112cm/d_2025_11_15/run_02/20251116T041159Z', id

In [55]:
i = 0
data_id = gcn_dr_match[i].dataId

In [34]:
template = butler.get("template_matched", dataId=data_id)
science = butler.get("preliminary_visit_image", dataId=data_id)
diffim = butler.get("difference_image", dataId=data_id)

In [21]:
dia_src = butler.get("dia_source_unfiltered", dataId=data_id).asAstropy()
good_dia_src = good_src(dia_src)

In [22]:
good_dia_src

NameError: name 'good_dia_source' is not defined

In [23]:
afwDisplay.setDefaultBackend("firefly")

In [24]:
transparency = 100

In [35]:
afw_display = afwDisplay.Display(frame=1)
afw_display.setMaskTransparency(transparency)
afw_display.scale("asinh", -20, 500)
# afw_display.scale("linear", "zscale")

afw_display.mtv(template)

In [36]:
afw_display = afwDisplay.Display(frame=2)
afw_display.setMaskTransparency(transparency)
afw_display.scale("asinh", -20, 50)
# afw_display.scale("linear", "zscale")

afw_display.mtv(science)

In [37]:
afw_display = afwDisplay.Display(frame=3)
afw_display.setMaskTransparency(transparency)
# afw_display.scale("asinh", -20, 50)
afw_display.scale("linear", "zscale")

afw_display.mtv(diffim.maskedImage)

In [30]:
# https://dp1.lsst.io/tutorials/notebook/103/notebook-103-5.html
def plot_points_on_image(xy, size=20, ctype="orange"):
    with afw_display.Buffering():
        for xi, yi in xy:
            afw_display.dot('o', xi, yi, size=size, ctype=ctype)

In [ ]:
plot_points_on_image(zip(good_dia_src["slot_Centroid_x"], good_dia_src["slot_Centroid_y"]), ctype="green")

In [38]:
wcs = diffim.getWcs()
candidate_coord = [SpherePoint(r*degrees, d*degrees) for r, d in zip(gcn_table["ra"], gcn_table["dec"])]
candidate_xy = wcs.skyToPixel(candidate_coord)

In [39]:
plot_points_on_image(candidate_xy, ctype="orange")

# These blue circles are wront for some reason I don't understand yet
# plot_points_on_image(zip(dia_src_match["slot_Centroid_x"][i:i+1], dia_src_match["slot_Centroid_y"][i:i+1]), ctype="blue")